<a href="https://colab.research.google.com/github/SasikiranJ/Coursera_Capstone/blob/master/segmentation%20and%20clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part:1  Importing Libraries

In [0]:
from pandas import DataFrame

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


# Using pandas built-in function to get table data frim wikipedia

In [0]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# To convert each extracted data into list of lists
df = data.values.tolist()


# To delete unnecessary rows and fill in some values

In [0]:
# Using function to convert list of lists obtained above to dataframe
df = DataFrame.from_records(data)

#To put columns
df.columns = ['PostalCode','Borough','Neighbourhood']

In [0]:
#To extract only rows with Borough values not equal to Not assigned as per condition
df_orig = df[df['Borough'] != 'Not assigned']

In [0]:
# Joining rows with same postal codes
dup = df_orig.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [0]:
# If Borough value present and Neighbourhood not present we have to replace Neighbourhood value with Borough value

mask = list((dup["Neighbourhood"] == "Not assigned"))

dup.iloc[mask.index(True),2]  = dup.iloc[mask.index(True),1]

In [0]:
# To print dimensions of cleaned dataframe  
print(dup.shape)

(103, 3)


In [0]:
dup

# Part:2 merging latitude and longitudinal values to dataset

In [0]:
!wget https://cocl.us/Geospatial_data

In [0]:
lat_long = pd.read_csv("Geospatial_data")

In [0]:
lat_long.columns = ["PostalCode","Latitude","Longitude"]
lat_long.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [0]:
dup1 = dup

dfinal = dup1.merge(lat_long, on="PostalCode", how = 'inner')

dfinal.shape

(103, 5)

# Part:3 Analysing data and clustering them with k-means

# Taking only Boroughs with Toronto

In [0]:

TorontoData = dfinal.loc[dfinal["Borough"].str.contains("Toronto")].reset_index(drop=True)

TorontoData.columns = ["PostalCode","Borough","Neighborhood","Latitude","Longitude"]

TorontoData.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# Getting coordinates for Toronto

In [0]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [0]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# Giving credentials to Authenticate with FourSquare

In [0]:
CLIENT_ID = 'FZX50I1NPLWBHJAIONNWRLS0LJDTCOMYPGCBIZDDKFHYN40T' # your Foursquare ID
CLIENT_SECRET = 'O4KI11NOHQYINMB2W3G1YPVXGHD341PNVOD3H1YAKWAJP2D2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZX50I1NPLWBHJAIONNWRLS0LJDTCOMYPGCBIZDDKFHYN40T
CLIENT_SECRET:O4KI11NOHQYINMB2W3G1YPVXGHD341PNVOD3H1YAKWAJP2D2


In [0]:
TorontoData.loc[0, 'Neighborhood']

'The Beaches'

In [0]:
neighborhood_latitude = TorontoData.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorontoData.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorontoData.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [0]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FZX50I1NPLWBHJAIONNWRLS0LJDTCOMYPGCBIZDDKFHYN40T&client_secret=O4KI11NOHQYINMB2W3G1YPVXGHD341PNVOD3H1YAKWAJP2D2&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [0]:
results = requests.get(url).json()
results

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


# Acquiring venues for each and every neighbourhood

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
Toronto_venues = getNearbyVenues(names=TorontoData['Neighborhood'],
                                   latitudes=TorontoData['Latitude'],
                                   longitudes=TorontoData['Longitude']
                                  )

print(Toronto_venues.shape)
Toronto_venues.head()


In [0]:
Toronto_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))


Indian = Toronto_venues.loc[Toronto_venues["Venue Category"].str.contains("Restaurant")].reset_index()

Indian

# encoding with 0 and 1 for categories of venues

In [0]:

# one hot encoding
Indian_onehot = pd.get_dummies(Indian[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Indian_onehot['Neighborhood'] = Indian['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Indian_onehot.columns[-1]] + list(Indian_onehot.columns[:-1])
Indian_onehot = Indian_onehot[fixed_columns]

Indian_onehot

In [0]:
Indian_onehot.shape

(417, 49)

In [0]:
Indian_grouped = Indian_onehot.groupby('Neighborhood').mean().reset_index()
Indian_grouped

In [0]:
Indian_grouped.shape

(32, 49)

In [0]:
num_top_venues = 5

for hood in Indian_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Indian_grouped[Indian_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Indian_grouped['Neighborhood']

for ind in np.arange(Indian_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Indian_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Thai Restaurant,American Restaurant,Asian Restaurant,Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Mediterranean Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant
1,Berczy Park,Seafood Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Eastern European Restaurant,Restaurant,Greek Restaurant,Belgian Restaurant,Thai Restaurant,Cajun / Creole Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Italian Restaurant,Caribbean Restaurant,Falafel Restaurant,Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
4,"Cabbagetown, St. James Town",Italian Restaurant,Restaurant,Japanese Restaurant,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Taiwanese Restaurant,Thai Restaurant,Belgian Restaurant,Eastern European Restaurant


In [0]:
# set number of clusters
kclusters = 5

Indian_grouped_clustering = Indian_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Indian_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 1, 0, 2, 3, 0, 2, 2], dtype=int32)

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Indian_merged = TorontoData

Indian_merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [0]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Indian_merged = Indian_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


Indian_merged1 = Indian_merged[np.isfinite(Indian_merged['Cluster Labels'])]

Indian_merged1["Cluster Labels"]  = Indian_merged1["Cluster Labels"].astype(int)



Indian_merged1.head() # check the last columns!

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,American Restaurant,Caribbean Restaurant,Restaurant,Indian Restaurant,Dumpling Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Thai Restaurant,Seafood Restaurant,Latin American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Gluten-free Restaurant,French Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Japanese Restaurant,Seafood Restaurant,Greek Restaurant,Doner Restaurant,French Restaurant


In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Indian_merged1['Latitude'], Indian_merged1['Longitude'], Indian_merged1['Neighborhood'], Indian_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
# cluster 1

Indian_merged1.loc[Indian_merged1['Cluster Labels'] == 0, Indian_merged1.columns[[1] + list(range(5, Indian_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,0,Italian Restaurant,Restaurant,Japanese Restaurant,Chinese Restaurant,Caribbean Restaurant,Indian Restaurant,Taiwanese Restaurant,Thai Restaurant,Belgian Restaurant,Eastern European Restaurant
13,Downtown Toronto,0,Mexican Restaurant,Restaurant,Italian Restaurant,French Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
19,Downtown Toronto,0,Italian Restaurant,Restaurant,Mexican Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Sushi Restaurant,Seafood Restaurant,Chinese Restaurant,New American Restaurant
25,Downtown Toronto,0,Japanese Restaurant,Restaurant,Italian Restaurant,Sushi Restaurant,French Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,American Restaurant
30,Downtown Toronto,0,Italian Restaurant,Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
33,West Toronto,0,Italian Restaurant,Caribbean Restaurant,Falafel Restaurant,Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
35,West Toronto,0,Italian Restaurant,Restaurant,Eastern European Restaurant,Cuban Restaurant,Belgian Restaurant,Brazilian Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant
36,West Toronto,0,Italian Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Latin American Restaurant,Restaurant,Falafel Restaurant,Doner Restaurant,Gluten-free Restaurant,Filipino Restaurant


In [0]:
#cluster 2
Indian_merged1.loc[Indian_merged1['Cluster Labels'] == 1, Indian_merged1.columns[[1] + list(range(5, Indian_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Restaurant,Vietnamese Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
37,East Toronto,1,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [0]:
#cluster 3
Indian_merged1.loc[Indian_merged1['Cluster Labels'] == 2, Indian_merged1.columns[[1] + list(range(5, Indian_merged1.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Italian Restaurant,American Restaurant,Caribbean Restaurant,Restaurant,Indian Restaurant,Dumpling Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant
3,East Toronto,2,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Thai Restaurant,Seafood Restaurant,Latin American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Gluten-free Restaurant,French Restaurant
7,Central Toronto,2,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Japanese Restaurant,Seafood Restaurant,Greek Restaurant,Doner Restaurant,French Restaurant
9,Central Toronto,2,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
12,Downtown Toronto,2,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant
15,Downtown Toronto,2,Restaurant,Italian Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,New American Restaurant,Fast Food Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant
16,Downtown Toronto,2,Seafood Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Eastern European Restaurant,Restaurant,Greek Restaurant,Belgian Restaurant,Thai Restaurant,Cajun / Creole Restaurant
17,Downtown Toronto,2,Italian Restaurant,Indian Restaurant,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Portuguese Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Middle Eastern Restaurant
18,Downtown Toronto,2,Thai Restaurant,American Restaurant,Asian Restaurant,Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Mediterranean Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant
20,Downtown Toronto,2,Italian Restaurant,Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,Chinese Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Greek Restaurant


In [0]:
#cluster 4
Indian_merged1.loc[Indian_merged1['Cluster Labels'] == 3, Indian_merged1.columns[[1] + list(range(5, Indian_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,3,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
6,Central Toronto,3,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
14,Downtown Toronto,3,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Modern European Restaurant,American Restaurant,Chinese Restaurant,Ethiopian Restaurant
26,Downtown Toronto,3,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Chinese Restaurant,Mexican Restaurant,Dumpling Restaurant,Caribbean Restaurant,Comfort Food Restaurant,Belgian Restaurant,Doner Restaurant,Filipino Restaurant
31,West Toronto,3,Fast Food Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
34,West Toronto,3,Mexican Restaurant,Italian Restaurant,Thai Restaurant,Cajun / Creole Restaurant,Fast Food Restaurant,Dumpling Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant


In [0]:
#cluster 5
Indian_merged1.loc[Indian_merged1['Cluster Labels'] == 4, Indian_merged1.columns[[1] + list(range(5, Indian_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Sushi Restaurant,Vietnamese Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


# Observations:-
     

## From above cluster 1 we can conclude that most of the restaurants are Italian.So we can suggest Investors  or entrepreneur to start Restaurants other than italian in downtown toronto or West Toronto ,so that there will be no competition.

## From Cluster 2 we can came to know that there are no italian restaurants in central toronto ,east toronto so entrepreneur start italian restaturants here.

## and most importantly it is best opportunity for entrepreneur who are willing to open indian restaurants , They can establish their restaurents anywhere in toronto region because there is little or no competition.